##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 語意檢索器快速入門指南


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/docs/semantic_retriever"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 ai.google.dev 上查看</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/docs/semantic_retriever.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/docs/semantic_retriever.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上查看原始檔</a>
  </td>
</table>


## 概述

大型語言模型 (LLM) 無需直接針對新能力進行訓練即可學習，但已知 LLM 在受指示提供未受訓回答問題時會「出現幻覺」。這部分是因為 LLM 不會察覺訓練後發生的事件，而且追溯 LLM 用來產生回應的來源也十分困難。要確保應用程式可靠且可擴充，LLM 提供的回應必須奠基於事實，並能夠引述其資訊來源。

為克服這些限制，有種常見的做法稱為「檢索增強生成」(RAG)，做法是透過「資訊檢索」(IR) 機制，增強傳送至 LLM 的提示，資料則從外部知識庫檢索而得。知識庫可以是你自己的文件、資料庫或 API 文件集。

本筆記本會引導你完成一個工作流程，以透過加強 LLM 的外部文件知識，並執行語義資訊檢索來回答問題，進而改善 LLM 的回應，問題回答方式則採用 [生成語言 API](../api/python/google/ai/generativelanguage) 的語義檢索器和屬性式問答 (AQA) API。

注意：這個 API 目前處於 [測試版](/docs/api_versions)，而且 [僅限特定區域](../available_regions) 使用。


## 設定


### 匯入生成式語言 API


In [ ]:
# Install the Client library (Semantic Retriever is only supported for versions >0.4.0)
!pip install -U google.ai.generativelanguage

## 驗證

語意檢索 API 讓你對自己的資料執行語意搜尋。由於這是 **你的資料** ，因此這需要更嚴格的存取控制，而不是 API 金鑰。使用服務帳戶透過 OAuth 進行驗證 ([#scrollTo=eLjhFIOQ7_Dk](#scrollTo=eLjhFIOQ7_Dk))，或是透過你的 [使用者身分證明](#scrollTo=9YGv4x9ehLba)。

這個快速入門使用簡易的驗證方式，適用於測試環境，而服務帳戶的設定通常比較容易上手。若要使用於生產環境，請在選擇適合你應用程式的 [存取身分證明](#scrollTo=9YGv4x9ehLba) 之前，先深入了解 [驗證和授權](https://developers.google.com/workspace/guides/auth-overview){:.external}。


### 使用服務帳戶設定 OAuth

按照下列步驟使用服務帳戶設定 OAuth：


1. 啟用 [Generative Language API](https://console.cloud.google.com/flows/enableapi?apiid=generativelanguage.googleapis.com){:.external}。

<img width=400 src="https://ai.google.dev/tutorials/images/semantic_retriever_enable_api.png">

2. 按照 [文件](https://developers.google.com/identity/protocols/oauth2/service-account#creatinganaccount){:.external} 建立服務帳戶。

 * 建立服務帳戶後，產生一個服務帳戶金鑰。

<img width=400 src="https://ai.google.dev/tutorials/images/semantic_retriever_service_account.png">

3. 使用左側邊欄中的檔案圖示，然後使用載入圖示，載入你的服務帳戶檔案，如下圖所示。

 * 將載入的檔案重新命名為 `service_account_key.json` 或變更下面程式碼中的變數 `service_account_file_name`。

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">


In [ ]:
!pip install -U google-auth-oauthlib

In [ ]:
service_account_file_name = 'service_account_key.json'

from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(service_account_file_name)

scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/generative-language.retriever'])

使用服務帳戶認證初始化客戶端函式庫。


In [ ]:
import google.ai.generativelanguage as glm
generative_service_client = glm.GenerativeServiceClient(credentials=scoped_credentials)
retriever_service_client = glm.RetrieverServiceClient(credentials=scoped_credentials)
permission_service_client = glm.PermissionServiceClient(credentials=scoped_credentials)

## 建立語料庫

使用「語意檢索應用程式介面 (API)」，每個專案最多可以定義 5 個自訂文字語料庫。定義語料庫時，你可以指定下列任何欄位：

  * `name`：`語料庫`資源名稱 (ID)。只能包含最多 40 個英數字元。如果建立時未輸入 `name`，系統將從 `display_name` 的字首和 12 個字元的隨機字尾產生長度不超過 40 個字元的獨一無二名稱。
  * `display_name`：`語料庫` 的人機可讀顯示名稱。只能包含最多 512 個字元，包括英數字、空格和破折號。


In [ ]:
example_corpus = glm.Corpus(display_name="Google for Developers Blog")
create_corpus_request = glm.CreateCorpusRequest(corpus=example_corpus)

# Make the request
create_corpus_response = retriever_service_client.create_corpus(create_corpus_request)

# Set the `corpus_resource_name` for subsequent sections.
corpus_resource_name = create_corpus_response.name
print(create_corpus_response)

### 取得已建立的語料庫

使用 `GetCorpusRequest` 方法以程式存取上文建立的 `語料庫`。 `name` 參數的值是指 `語料庫` 的完整資源名稱，並在上方Cell設為 `corpus_resource_name`。預期的格式為 `corpora/corpus-123`。


In [ ]:
get_corpus_request = glm.GetCorpusRequest(name=corpus_resource_name)

# Make the request
get_corpus_response = retriever_service_client.get_corpus(get_corpus_request)

# Print the response
print(get_corpus_response)

## 建立文件

`Corpus` 最多可包含 10,000 份 `Document`。你可在定義文件時指定下列任一欄位：

  * `name`：`Document` 資源名稱 (ID)。僅能包含最多 40 個字元 (僅英數字或破折號)。ID 不能以破折號開頭或結尾。如果在建立時名稱為空，將由 `display_name` 衍生一個獨特的名稱以及 12 個字元的隨機後綴。
  * `display_name`：人類可讀顯示名稱。僅能包含最多 512 個字元，包括英數字、空白和破折號。

`Document` 也支援最多 20 個使用者指定的 `custom_metadata` 欄位，指定為關鍵字值配對。自訂元資料可能是字串、字串清單或數字。請注意，字串清單最多可支援 10 個值，而數字值則在 API 中以浮點數表示。


In [ ]:
# Create a document with a custom display name.
example_document = glm.Document(display_name="Introducing Project IDX, An Experiment to Improve Full-stack, Multiplatform App Development")

# Add metadata.
# Metadata also supports numeric values not specified here
document_metadata = [
    glm.CustomMetadata(key="url", string_value="https://developers.googleblog.com/2023/08/introducing-project-idx-experiment-to-improve-full-stack-multiplatform-app-development.html")]
example_document.custom_metadata.extend(document_metadata)

# Make the request
# corpus_resource_name is a variable set in the "Create a corpus" section.
create_document_request = glm.CreateDocumentRequest(parent=corpus_resource_name, document=example_document)
create_document_response = retriever_service_client.create_document(create_document_request)

# Set the `document_resource_name` for subsequent sections.
document_resource_name = create_document_response.name
print(create_document_response)

### 取得建立的的文件

使用 `GetDocumentRequest` 方法以程式設計存取你在上面建立的文件。`name` 參數的值參照文件資源的完整名稱，並設定於上面的Cell為 `document_resource_name`。預期的格式是 `corpora/corpus-123/documents/document-123`。


In [ ]:
get_document_request = glm.GetDocumentRequest(name=document_resource_name)

# Make the request
# document_resource_name is a variable set in the "Create a document" section.
get_document_response = retriever_service_client.get_document(get_document_request)

# Print the response
print(get_document_response)

## 擷取和分塊文件

為了提升語義檢索時向量資料庫所返回內容的相關性，在擷取文件的同時將大型文件分割成較小的片段或 **塊** 。

```chunk``` 是 ```document``` 的子部分，在向量表示和儲存的目的上被視為獨立的單位。```chunk``` 最多可以擁有 2043 個 Token。```corpus``` 最多可以擁有 100 萬個 ```chunk```。

與 ```Document``` 類似，```Chunk``` 也支援使用者自訂的 ```custom_metadata``` 欄位，最多 20 個，指定為關鍵字值對的形式。使用者自訂的欄位可以是字串、字串清單或數字。請注意，字串清單最多可以支援 10 個值，而數字值在 API 中會表示為浮點數。

本指南使用了 Google 的 [Open Source HtmlChunker](https://github.com/google/labs-prototypes/tree/main/seeds/chunker-python){:.external}。

其他可使用的分塊器包含 [LangChain](https://python.langchain.com/docs/get_started/introduction){:.external} 或 [LlamaIndex](https://www.llamaindex.ai/){:.external}。


### HtmlChunker 處理進出的 HTML 及區塊


In [ ]:
!pip install google-labs-html-chunker

from google_labs_html_chunker.html_chunker import HtmlChunker

from urllib.request import urlopen

取得網站的 HTML DOM。在此，HTML 是直接讀取的，但最好在渲染後取得 HTML 以包含 Javascript 注入的 HTML，例如 `document.documentElement.innerHTML`。


In [ ]:
with(urlopen("https://developers.googleblog.com/2023/08/introducing-project-idx-experiment-to-improve-full-stack-multiplatform-app-development.html")) as f:
  html = f.read().decode("utf-8")

將純文字文件拆解成段落，並從這些段落建立 `Chunk`。此步驟建立 `Chunk` 物件，而下一步會將它們上傳到語意化檢索 API。


In [ ]:
# Chunk the file using HtmlChunker
chunker = HtmlChunker(
    max_words_per_aggregate_passage=200,
    greedily_aggregate_sibling_nodes=True,
    html_tags_to_exclude={"noscript", "script", "style"},
)
passages = chunker.chunk(html)
print(passages)


# Create `Chunk` entities.
chunks = []
for passage in passages:
    chunk = glm.Chunk(data={'string_value': passage})
    # Optionally, you can add metadata to a chunk
    chunk.custom_metadata.append(glm.CustomMetadata(key="tags",
                                                    string_list_value=glm.StringList(
                                                        values=["Google For Developers", "Project IDX", "Blog", "Announcement"])))
    chunk.custom_metadata.append(glm.CustomMetadata(key="chunking_strategy",
                                                    string_value="greedily_aggregate_sibling_nodes"))
    chunk.custom_metadata.append(glm.CustomMetadata(key = "publish_date",
                                                    numeric_value = 20230808))
    chunks.append(chunk)
print(chunks)

## 批量建立塊

分批建立塊。 你可以為每個批次要求指定最多 100 個區塊。

對於單一塊建立，請使用 `CreateChunk()`。


In [ ]:
# Option 1: Use HtmlChunker in the section above.
# `chunks` is the variable set from the section above.
create_chunk_requests = []
for chunk in chunks:
  create_chunk_requests.append(glm.CreateChunkRequest(parent=document_resource_name, chunk=chunk))

# Make the request
request = glm.BatchCreateChunksRequest(parent=document_resource_name, requests=create_chunk_requests)
response = retriever_service_client.batch_create_chunks(request)
print(response)

或者，你可以使用 HtmlChunker 來製作區塊。


In [ ]:
# Add up to 100 CreateChunk requests per batch request.
# document_resource_name is a variable set in the "Create a document" section.
chunks = []
chunk_1 = glm.Chunk(data={'string_value': "Chunks support user specified metadata."})
chunk_1.custom_metadata.append(glm.CustomMetadata(key="section",
                                                  string_value="Custom metadata filters"))
chunk_2 = glm.Chunk(data={'string_value': "The maximum number of metadata supported is 20"})
chunk_2.custom_metadata.append(glm.CustomMetadata(key = "num_keys",
                                                  numeric_value = 20))
chunks = [chunk_1, chunk_2]
create_chunk_requests = []
for chunk in chunks:
  create_chunk_requests.append(glm.CreateChunkRequest(parent=document_resource_name, chunk=chunk))

# Make the request
request = glm.BatchCreateChunksRequest(parent=document_resource_name, requests=create_chunk_requests)
response = retriever_service_client.batch_create_chunks(request)
print(response)

### 列出 `Chunk` 並取得狀態

使用 `ListChunksRequest` 方法以分頁列表取得所有可用的 `Chunk`，每個頁面最多有 100 個 `Chunk`，並根據 `Chunk.create_time` 的升冪進行排序。如果你未指定限制，則最多會傳回 10 個 `Chunk`。

提供在 `ListChunksRequest` 回應中傳回的 `next_page_token` 做為下一個請求的參數，以擷取下一個頁面。請注意，在分頁時，提供給 `ListChunks` 的所有其他參數都必須與提供頁面權杖的呼叫相符。

所有 `Chunk` 都會傳回 `狀態`。在查詢 `Corpus` 之前，使用這個狀態檢查 `Chunks` 的狀態。`Chunk` 狀態包括 - `未指定`、`處理中`、`已啟用` 和 `失敗`。你只能查詢 `已啟用` 的 `Chunk`。


In [ ]:
# Make the request
request = glm.ListChunksRequest(parent=document_resource_name)
list_chunks_response = retriever_service_client.list_chunks(request)
for index, chunks in enumerate(list_chunks_response.chunks):
  print(f'\nChunk # {index + 1}')
  print(f'Resource Name: {chunks.name}')
  # Only ACTIVE chunks can be queried.
  print(f'State: {glm.Chunk.State(chunks.state).name}')

## 擷取另一份文件

經由 HtmlChunker 新增另一份 `Document`，並新增篩選器。


In [ ]:
# Create a document with a custom display name.
example_document = glm.Document(display_name="How it’s Made: Interacting with Gemini through multimodal prompting")

# Add document metadata.
# Metadata also supports numeric values not specified here
document_metadata = [
    glm.CustomMetadata(key="url", string_value="https://developers.googleblog.com/2023/12/how-its-made-gemini-multimodal-prompting.html")]
example_document.custom_metadata.extend(document_metadata)

# Make the CreateDocument request
# corpus_resource_name is a variable set in the "Create a corpus" section.
create_document_request = glm.CreateDocumentRequest(parent=corpus_resource_name, document=example_document)
create_document_response = retriever_service_client.create_document(create_document_request)

# Set the `document_resource_name` for subsequent sections.
document_resource_name = create_document_response.name
print(create_document_response)

# Chunks - add another webpage from Google for Developers
with(urlopen("https://developers.googleblog.com/2023/12/how-its-made-gemini-multimodal-prompting.html")) as f:
  html = f.read().decode("utf-8")

# Chunk the file using HtmlChunker
chunker = HtmlChunker(
    max_words_per_aggregate_passage=100,
    greedily_aggregate_sibling_nodes=False,
)
passages = chunker.chunk(html)

# Create `Chunk` entities.
chunks = []
for passage in passages:
    chunk = glm.Chunk(data={'string_value': passage})
    chunk.custom_metadata.append(glm.CustomMetadata(key="tags",
                                                    string_list_value=glm.StringList(
                                                        values=["Google For Developers", "Gemini API", "Blog", "Announcement"])))
    chunk.custom_metadata.append(glm.CustomMetadata(key="chunking_strategy",
                                                    string_value="no_aggregate_sibling_nodes"))
    chunk.custom_metadata.append(glm.CustomMetadata(key = "publish_date",
                                                    numeric_value = 20231206))
    chunks.append(chunk)

# Make the request
create_chunk_requests = []
for chunk in chunks:
  create_chunk_requests.append(glm.CreateChunkRequest(parent=document_resource_name, chunk=chunk))
request = glm.BatchCreateChunksRequest(parent=document_resource_name, requests=create_chunk_requests)
response = retriever_service_client.batch_create_chunks(request)
print(response)

## 查詢語料庫

使用 `QueryCorpusRequest` 方法進行語意搜尋以獲取相關章節。

* `results_count`: 指定要傳回的章節數目。最大值為 100。如果未指定，API 會傳回最多 10 個 `Chunk`。
* `metadata_filters`: 依據 `chunk_metadata` 或 `document_metadata` 篩選。每個 `MetadataFilter` 都必須對應到一個唯一的金鑰。多個 `MetadataFilter` 物件會透過邏輯 `AND` 結合。類似元資料篩選條件會透過邏輯 `OR` 結合。以下是部分範例：

```
(year >= 2020 OR year < 2010) AND (genre = drama OR genre = action)

metadata_filter = [
  {
    key = "document.custom_metadata.year"
    conditions = [
      {int_value = 2020, operation = GREATER_EQUAL},
      {int_value = 2010, operation = LESS}]
  },
  {
    key = "document.custom_metadata.genre"
    conditions = [
      {string_value = "drama", operation = EQUAL},
      {string_value = "action", operation = EQUAL}}]
  }]
```

請注意，對於相同的金鑰，只有數字值支援「AND」。字串值只支援「OR」。

```
("Google for Developers" in tags) and (20230314 > publish_date)

metadata_filter = [
 {
    key = "chunk.custom_metadata.tags"
    conditions = [
    {string_value = 'Google for Developers', operation = INCLUDES},
  },
  {
    key = "chunk.custom_metadata.publish_date"
    conditions = [
    {numeric_value = 20230314, operation = GREATER_EQUAL}]
  }]
```


In [ ]:
user_query = "What is the purpose of Project IDX?"
results_count = 5

# Add metadata filters for both chunk and document.
chunk_metadata_filter = glm.MetadataFilter(key='chunk.custom_metadata.tags',
                                           conditions=[glm.Condition(
                                              string_value='Google For Developers',
                                              operation=glm.Condition.Operator.INCLUDES)])

# Make the request
# corpus_resource_name is a variable set in the "Create a corpus" section.
request = glm.QueryCorpusRequest(name=corpus_resource_name,
                                 query=user_query,
                                 results_count=results_count,
                                 metadata_filters=[chunk_metadata_filter])
query_corpus_response = retriever_service_client.query_corpus(request)
print(query_corpus_response)

## 歸因問答

使用 [`GenerateAnswer`](../api/python/google/ai/generativelanguage/GenerateAnswerRequest) 方法在你的文件、語料庫或一組段落中執行歸因問答。

歸因問答 (AQA) 指的是在給定的背景下回答問題並提供歸因，同時最大限度地減少幻覺。

在需要 AQA 的情況下，與使用未調整語言模型相比，`GenerateAnswer` 提供以下幾個優點：

* 底層模型經過訓練，只會返回基於所提供背景的答案。
* 它識別歸因 (為答案做出貢獻的所提供背景片段)。歸因使用戶能夠驗證答案。
* 它估計給定的 (問題、背景) 對的 `answerable_probability`，這進一步讓你能夠根據返回的答案在多大程度上是合理的並正確來轉變產品行為。

注意：AQA 目前僅支援英語查詢。


### `answerable_probability` 和「我不知道」問題

在某些情況下，對問題最好的回答實際上是「我不知道」。例如，如果提供的脈絡不包含問題的答案，則該問題被認為是「不可回答的」。

AQA 模型非常擅長識別此類情況。它甚至可以區分可回答性和不可回答性的程度。

然而，`GenerateAnswer` API 讓你的手握有最終決策權：

* *始終* 嘗試傳回有根據的答案 - 即使該答案相對不像是有根據且正確的。
* 傳回 `answerable_probability` 值 - 該模型對於答案有根據且正確的機率估計。

低 `answerable_probability` 可能可以由以下一個或多個因素說明：

* 模型不確定其答案是否正確。
* 模型不確定其答案是否建立在引用的段落上；答案可能取而代之根據普世知識得出。例如：`question="1 + 1 =?", passages=["2 + 2 = 4"]` → `answer = 2, answerable_probability = 0.02`
* 模型提供了與問題完全無關的資訊。例如：`question="這有我的尺寸嗎？, passages=["可用尺寸 5 - 11"]` → `answer="是的，有尺寸 5 - 11", answerable_probability = 0.03"`
* 在 GenerateAnswerRequest 中沒有提出良好的問題。

由於低 `answerable_probability` 表示 GenerateAnswerResponse.answer 可能錯誤或沒有根據，**強烈建議進一步處理回答，檢查 `answerable_probability`** 。

在 `answerable_probability` 較低時，有些客戶可能想要：
* 向使用者顯示類似於「我們無法回答那個問題」的訊息。
* 退回到從普世知識回答問題的通用 LLM。此類備援方案的閾值和性質將取決於個別使用案例。`answerable_probability` = 0.5 是良好的起點閾值。


### AQA 有用提示

如需完整 API 規格，請參閱 [`GenerateAnswerRequest` API 參考](../api/python/google/ai/generativelanguage/GenerateAnswerRequest)。

* *段落長度*: 建議每段落少於 300 個 Token。
* *段落排序*:
  * 如果你提供 `GenerateAnswerRequest.inline_passages`，則段落應以與查詢相關性遞減的順序排序。如果超出模型的上下文長度限制，最後 (最不相關) 的段落將被省略。
  * 如果你提供 `GenerateAnswerRequest.semantic_retriever`，則將自動為你執行相關性排序。
* *限制*: AQA 模型專門用於問答。對於其他使用案例，例如創意寫作、摘要等，請透過 GenerateContent 呼叫一般用途模型。
  * *聊天*: 如果使用者輸入已知是一個問題，可以從特定背景中得到解答，則 AQA 可以回答聊天疑問。但如果使用者輸入的可能是任何類型輸入，則一般用途模型可能是較好的選擇。
* *溫度*:
  * 通常，建議使用相對較低的溫度 (~0.2) 來執行精確的 AQA。
  * 如果你的使用案例依賴於決定性輸出，請將溫度設定為 0。


In [ ]:
user_query = "What is the purpose of Project IDX?"
answer_style = "ABSTRACTIVE" # Or VERBOSE, EXTRACTIVE
MODEL_NAME = "models/aqa"

# Make the request
# corpus_resource_name is a variable set in the "Create a corpus" section.
content = glm.Content(parts=[glm.Part(text=user_query)])
retriever_config = glm.SemanticRetrieverConfig(source=corpus_resource_name, query=content)
req = glm.GenerateAnswerRequest(model=MODEL_NAME,
                                contents=[content],
                                semantic_retriever=retriever_config,
                                answer_style=answer_style)
aqa_response = generative_service_client.generate_answer(req)
print(aqa_response)

In [ ]:
# Get the metadata from the first attributed passages for the source
chunk_resource_name = aqa_response.answer.grounding_attributions[0].source_id.semantic_retriever_chunk.chunk
get_chunk_response = retriever_service_client.get_chunk(name=chunk_resource_name)
print(get_chunk_response)

### 更多選項：使用內嵌段落 AQA

或者，你可以直接使用 AQA 端點，而無需透過傳遞 `inline_passages` 來使用語意檢索 API。


In [ ]:
user_query = "What is AQA from Google?"
user_query_content = glm.Content(parts=[glm.Part(text=user_query)])
answer_style = "VERBOSE" # or ABSTRACTIVE, EXTRACTIVE
MODEL_NAME = "models/aqa"

# Create the grounding inline passages
grounding_passages = glm.GroundingPassages()
passage_a = glm.Content(parts=[glm.Part(text="Attributed Question and Answering (AQA) refers to answering questions grounded to a given corpus and providing citation")])
grounding_passages.passages.append(glm.GroundingPassage(content=passage_a, id="001"))
passage_b = glm.Content(parts=[glm.Part(text="An LLM is not designed to generate content grounded in a set of passages. Although instructing an LLM to answer questions only based on a set of passages reduces hallucination, hallucination still often occurs when LLMs generate responses unsupported by facts provided by passages")])
grounding_passages.passages.append(glm.GroundingPassage(content=passage_b, id="002"))
passage_c = glm.Content(parts=[glm.Part(text="Hallucination is one of the biggest problems in Large Language Models (LLM) development. Large Language Models (LLMs) could produce responses that are fictitious and incorrect, which significantly impacts the usefulness and trustworthiness of applications built with language models.")])
grounding_passages.passages.append(glm.GroundingPassage(content=passage_c, id="003"))

# Create the request
req = glm.GenerateAnswerRequest(model=MODEL_NAME,
                                contents=[user_query_content],
                                inline_passages=grounding_passages,
                                answer_style=answer_style)
aqa_response = generative_service_client.generate_answer(req)
print(aqa_response)

## 共享語料庫

你可以選擇使用 [`CreatePermissionRequest`](../api/python/google/ai/generativelanguage/CreatePermissionRequest) API 與他人共享語料庫。

限制：

* 有 2 個共享角色：`READER` 和 `EDITOR`。
  * `READER` 可以查詢語料庫。
  * `WRITER` 擁有讀取權限，此外還可以編輯和共享語料庫。
* 語料庫可以對 `EVERYONE` 授予 `user_type` 讀取權限，使其公開。


In [ ]:
# Replace your-email@gmail.com with the email added as a test user in the OAuth Quickstart
shared_user_email = "TODO-your-email@gmail.com" #  @param {type:"string"}
user_type = "USER"
role = "READER"

# Make the request
# corpus_resource_name is a variable set in the "Create a corpus" section.
request = glm.CreatePermissionRequest(
    parent=corpus_resource_name,
    permission=glm.Permission(grantee_type=user_type,
                              email_address=shared_user_email,
                              role=role))
create_permission_response = permission_service_client.create_permission(request)
print(create_permission_response)

## 刪除語料庫

使用 [`DeleteCorpusRequest`](../api/python/google/ai/generativelanguage/DeleteCorpusRequest) 刪除使用者語料庫以及所有相關的 `Document` 和 `Chunk`。

請注意，非空白語料庫會擲回錯誤，而不會指定 `force=True` 標記。如果你設定 `force=True`，與此 `Document` 相關的任何 `Chunk` 和物件也會被刪除。

如果 `force=False` (預設值) 且 `Document` 包含任何 `Chunk`，將會傳回 `FAILED_PRECONDITION` 錯誤訊息。


In [ ]:
# Set force to False if you don't want to delete non-empty corpora.
req = glm.DeleteCorpusRequest(name=corpus_resource_name, force=True)
delete_corpus_response = retriever_service_client.delete_corpus(req)
print("Successfully deleted corpus: " + corpus_resource_name)

## 總結及延伸閱讀

本指南介紹了 Generative Language API 的語意檢索器和屬性化問答 (AQA) API，並展示了如何使用它對你的自訂文字資料進行語意資訊檢索。請注意，這個 API 也支援 [LlamaIndex](https://www.llamaindex.ai/){:.external} 資料架構。請參閱 [教學](https://github.com/run-llama/llama_index/blob/main/docs/examples/managed/GoogleDemo.ipynb){:.external} 以進一步瞭解。

也請參閱 [API 文件](https://ai.google.dev/api) 以進一步瞭解其他可用的功能。


## 附錄：使用使用者憑證設定 OAuth

從 [OAuth 快速入門](https://ai.google.dev/docs/oauth_quickstart) 按照以下步驟設定 OAuth 認證。

1. [設定 OAuth 同意畫面](https://ai.google.dev/docs/oauth_quickstart#configure-oauth)。

1. [授權桌面應用程式的憑證](https://ai.google.dev/docs/oauth_quickstart#authorize-credentials)。若要在 Colab 中執行這個筆記本，請先將你的憑證檔 (通常為 `client_secret_*.json`) 重新命名為 `client_secret.json`。然後使用左邊側欄上的檔案圖示，再按下上傳圖示，上傳檔案 (如下圖截圖所示)。

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">


In [ ]:
# Replace TODO-your-project-name with the project used in the OAuth Quickstart
project_name = "TODO-your-project-name" #  @param {type:"string"}
# Replace TODO-your-email@gmail.com with the email added as a test user in the OAuth Quickstart
email = "TODO-your-email@gmail.com" #  @param {type:"string"}
# Rename the uploaded file to `client_secret.json` OR
# Change the variable `client_file_name` in the code below.
client_file_name = "client_secret.json"

# IMPORTANT: Follow the instructions from the output - you must copy the command
# to your terminal and copy the output after authentication back here.
!gcloud config set project $project_name
!gcloud config set account $email

# NOTE: The simplified project setup in this tutorial triggers a "Google hasn't verified this app." dialog.
# This is normal, click "Advanced" -> "Go to [app name] (unsafe)"
!gcloud auth application-default login --no-browser --client-id-file=$client_file_name --scopes="https://www.googleapis.com/auth/generative-language.retriever,https://www.googleapis.com/auth/cloud-platform"

初始化用戶端資源庫並從 [建立語料庫](#scrollTo=4EQJD2PWD56T) 重新執行筆記本。


In [ ]:
import google.ai.generativelanguage as glm

generative_service_client = glm.GenerativeServiceClient()
retriever_service_client = glm.RetrieverServiceClient()
permission_service_client = glm.PermissionServiceClient()